In [ ]:
### import os
# folder='/content/drive/MyDrive/DATA_2/data'
# os.system(f'rm -rf {folder}')

#install packages

In [ ]:
!git clone https://github.com/ultralytics/yolov5  # clone repo
%cd yolov5

In [ ]:
# install dependencies as necessary
!pip install -qr requirements.txt  # install dependencies (ignore errors)
import torch

from IPython.display import Image, clear_output  # to display images
from utils.downloads import attempt_download  # to download models/datasets

In [ ]:
clear_output()
print('Setup complete. Using torch %s %s' % (torch.__version__, torch.cuda.get_device_properties(0) if torch.cuda.is_available() else 'CPU'))

In [ ]:
%pwd

##Download dataset

In [ ]:
#Download dataset
%cd /content/
!pwd
!pip install gdown
#extract id from url
url='https://drive.google.com/file/d/1vB91Dd4RWAzm9E9KUm80H8cbDufGI9xR/view?usp=drive_link'
# url=input("enter the url : ")
id=url.split('/')[-2]
print("id: ",id)
import gdown

# a file
# output= input("enter the output filename : ")
output= 'data.zip'
gdown.download(id=id, output=output)
import os
dataset_path='/content/dataset/'
zip_file='/content/'+output
os.makedirs(dataset_path,exist_ok=True)
!unzip {zip_file} -d {dataset_path}

In [ ]:
data_location=dataset_path+'data/data.yaml'
print(data_location)
train_location=dataset_path+'data/train'
print(train_location)
valid_location=dataset_path+'data/valid'
print(valid_location)
test_location=dataset_path+'data/test/images'
print(test_location)

/content/dataset/data/data.yaml
/content/dataset/data/train
/content/dataset/data/valid


In [ ]:
%cat {data_location}

# Train/val/test sets as 
## 1) dir: path/to/imgs, 2) file: path/to/imgs.txt, or 3) list: [path/to/imgs1, path/to/imgs2, ..]


#path: ../datasets/coco128  # dataset root dir
train: /content/dataset/data/train  # train images (relative to 'path') 128 images
val: /content/dataset/data/valid # val images (relative to 'path') 128 images
# test:  # test images (optional)

# Classes
nc: 6  # number of classes
names: ['Anthracnose','Bacterial Canker','Gall Midge','Healthy','Powdery Mildew','Scooty Mould']  # class names


##overwrite the data.yaml file

In [ ]:
from IPython.core.magic import register_line_cell_magic

@register_line_cell_magic
def writetemplate(line, cell):
    with open(line, 'w') as f:
        f.write(cell.format(**globals()))

In [ ]:
%%writetemplate {data_location}
# Train/val/test sets as
## 1) dir: path/to/imgs, 2) file: path/to/imgs.txt, or 3) list: [path/to/imgs1, path/to/imgs2, ..]


#path: ../datasets/coco128  # dataset root dir
train: {train_location}  # train images (relative to 'path') 128 images
val: {valid_location} # val images (relative to 'path') 128 images
# test:  # test images (optional)

# Classes
nc: 6  # number of classes
names: ['Anthracnose','Bacterial Canker','Gall Midge','Healthy','Powdery Mildew','Scooty Mould']  # class names

In [ ]:
%cat {data_location}

In [ ]:
import yaml
with open(data_location, 'r') as stream:
    num_classes = str(yaml.safe_load(stream)['nc'])

In [ ]:
#get the num_of classes from data.yaml file
num_classes

In [ ]:
%cat /content/yolov5/models/yolov5m.yaml

In [ ]:
from IPython.core.magic import register_line_cell_magic

@register_line_cell_magic
def writetemplate(line, cell):
    with open(line, 'w') as f:
        f.write(cell.format(**globals()))

##overwrite the yolov5 model yaml file

In [ ]:
%%writetemplate /content/yolov5/models/custom_yolov5m.yaml

# Parameters
nc: {num_classes} # number of classes
depth_multiple: 0.67 # model depth multiple
width_multiple: 0.75 # layer channel multiple
anchors:
  - [10, 13, 16, 30, 33, 23] # P3/8
  - [30, 61, 62, 45, 59, 119] # P4/16
  - [116, 90, 156, 198, 373, 326] # P5/32

# YOLOv5 v6.0 backbone
backbone:
  # [from, number, module, args]
  [
    [-1, 1, Conv, [64, 6, 2, 2]], # 0-P1/2
    [-1, 1, Conv, [128, 3, 2]], # 1-P2/4
    [-1, 3, C3, [128]],
    [-1, 1, Conv, [256, 3, 2]], # 3-P3/8
    [-1, 6, C3, [256]],
    [-1, 1, Conv, [512, 3, 2]], # 5-P4/16
    [-1, 9, C3, [512]],
    [-1, 1, Conv, [1024, 3, 2]], # 7-P5/32
    [-1, 3, C3, [1024]],
    [-1, 1, SPPF, [1024, 5]], # 9
  ]

# YOLOv5 v6.0 head
head: [
    [-1, 1, Conv, [512, 1, 1]],
    [-1, 1, nn.Upsample, [None, 2, "nearest"]],
    [[-1, 6], 1, Concat, [1]], # cat backbone P4
    [-1, 3, C3, [512, False]], # 13

    [-1, 1, Conv, [256, 1, 1]],
    [-1, 1, nn.Upsample, [None, 2, "nearest"]],
    [[-1, 4], 1, Concat, [1]], # cat backbone P3
    [-1, 3, C3, [256, False]], # 17 (P3/8-small)

    [-1, 1, Conv, [256, 3, 2]],
    [[-1, 14], 1, Concat, [1]], # cat head P4
    [-1, 3, C3, [512, False]], # 20 (P4/16-medium)

    [-1, 1, Conv, [512, 3, 2]],
    [[-1, 10], 1, Concat, [1]], # cat head P5
    [-1, 3, C3, [1024, False]], # 23 (P5/32-large)

    [[17, 20, 23], 1, Detect, [nc, anchors]], # Detect(P3, P4, P5)
  ]

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

###Select the model variant

In [ ]:
MODEL='yolov5m.pt'

###define output path

In [ ]:
model=MODEL.split('.')[0]
print(model)
output_path=f'/content/drive/MyDrive/{model}/output'
os.makedirs(output_path,exist_ok=True)
%cd {output_path}

In [ ]:
model_yaml='/content/yolov5/models/custom_yolov5m.yaml'


#Train the model

In [ ]:
# train yolov5s on custom data for 200 epochs
# time its performance
%%time
clear_output()
!python /content/yolov5/train.py --img 416 --batch 16 --epochs 100 --data {data_location} --cfg {model_yaml} --weights {model_weight} --name yolov5m_results  --cache

#Display results

In [ ]:
# result_path='/content/yolov5/runs/train/yolov5s_results3'
result_path=input('Enter the result path : ')
result_path='/content/yolov5/'+result_path
print(result_path)

In [ ]:
import os

In [ ]:
from utils.plots import plot_results  # plot results.txt as results.png
Image(filename=os.path.join(result_path,'results.png'), width=1000)

In [ ]:
Image(filename=os.path.join(result_path,'val_batch0_labels.jpg'), width=900)

In [ ]:
print("GROUND TRUTH AUGMENTED TRAINING DATA:")
Image(filename=os.path.join(result_path,'train_batch0.jpg'), width=900)

In [ ]:
# trained weights are saved by default in our weights folder
%ls runs/

In [ ]:
%ls runs/train/yolov5s_results2/weights

#Test the pre-trained model

In [ ]:
Weights=os.path.join(result_path,'weights/best.pt')
# Weights='/content/yolov5/runs/train/yolov5s_results/weights/best.pt'
Weights

In [ ]:
%cd /content/yolov5/
!python detect.py --weights {Weights} --img 416 --conf 0.25 --source {Test_images}

#Display the predicted images

In [ ]:
# detect_path='/content/yolov5/runs/detect/exp2'
detect_path=input("enter the exp path : ")
detect_path='/content/yolov5/'+detect_path
images_path=detect_path+'/*.jpg'
images_path

In [ ]:
import glob
from IPython.display import Image, display

for imageName in glob.glob(images_path): #assuming JPG
    display(Image(filename=imageName))

#Display the tensorboard result


In [ ]:
%load_ext tensorboard
%tensorboard --logdir {output_path}